In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment4.ipynb")

# Assignment 4: Accessing JSON Data with APIs

Many data providers publish application programming interface (API) for users to access their data. In this assignment we will use the National Aeronautics and Space Administration (NASA) API to access their [NeoWs (Near Earth Object Web Service) service](https://data.nasa.gov/Space-Science/Asteroids-NeoWs-API/73uw-d9i8).

We will prepare to gather data about asteroids using the NASA API. 

## API Key

[Get your API key](https://api.nasa.gov) and save it as a variable named `my_api_key`. _Warning: API keys are like passwords. In practice, API key should be kept private. However, in this assignment, we will check if your key works._

To check if your API key works, construct a query. Consider the following query that uses a `DEMO_KEY`:
```
https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key=DEMO_KEY
```
To use your key instead of the temporary key, replace `DEMO_KEY` with the _value of `my_api_key`_ variable defined earlier. To do that you can replace `DEMO_KEY` with `{my_api_key}`, which tells ipython to replace the term with the value of the variable `my_api_key`.

Assign the query result to `check_key` variable.

In [ ]:
my_api_key = ...
check_key = !curl -s ... # the query URL with your API key goes here
# inspect the contents of `check_key` but no need to show it in the notebook

In [ ]:
grader.check("my_api_key")

<!-- BEGIN QUESTION -->

## Download `jq` for use

`jq` is a lightweight and flexible command-line JSON processor. It is like `sed` for JSON data - you can use it to slice and filter and map and transform structured data with the same ease that `sed`, `awk`, `grep` and friends let you play with text.

Applications such as `jq` are often a single executable file. `jq` can be downloaded from the project's [GitHub page](https://github.com/jqlang/jq/releases/tag/jq-1.7). Specifically, you will download the binary `jq-linux-amd64`. The downloaded file can be set as "executable" (`x`) for the user (`u`) with the `chmod` command.

Command line applications are "installed" by placing the executable file in a searchable location as defined in `PATH` environment variable. 

The `PATH` environment variable is a system variable that operating systems use to locate executables from the command line or Terminal window. `PATH` is essentially a list of directory paths. When you type a command to run, the system looks for it in the directories specified by `PATH`.

Fill-in the code below to download `jq` and place it in searchable directory, `/opt/conda/bin`. In other words, replace the following placeholders and remove the comment character `#`.
1. `[SEARCHABLE_DIRECTORY]`
1. `[JQ_DOWNLOAD_LINK]`

In [ ]:
%%bash
# wget -q [JQ_DOWNLOAD_LINK] -O [SEARCHABLE_DIRECTORY]/jq
# chmod u+x [SEARCHABLE_DIRECTORY]/jq
echo "Location of installed jq: $(which jq)"
echo "Installed jq version: $(jq --version)"

<!-- END QUESTION -->

## Quickstart with `jq`

We will use the following data to illustrate key features of `jq`. Running the following cell will create a file `made_up_file.json`.

In [ ]:
%%writefile made_up_file.json
{
  "company": "Big Data Inc.",
  "employees": [
    {
      "firstName": "John",
      "lastName": "Doe",
      "skills": [
        "Python",
        "Java",
        "C++"
      ]
    },
    {
      "firstName": "Anna",
      "lastName": "Smith",
      "skills": [
        "JavaScript",
        "HTML",
        "CSS"
      ]
    },
    {
      "firstName": "Peter",
      "lastName": "Jones",
      "skills": [
        "Python",
        "R",
        "SQL"
      ]
    }
  ]
}

Run and understand the following commands to learn what each does:

In [ ]:
!jq '.company' made_up_file.json # extract company attribute

In [ ]:
!jq '.employees[1]' made_up_file.json # extract second employee

In [ ]:
!jq '.employees[].skills[0]' made_up_file.json # extract first skill of each employee

In [ ]:
!jq '.employees[] | .firstName' made_up_file.json # using pipe makes some things easier

In [ ]:
!jq '.employees[] | { first_name: .firstName, last_name: .lastName }' made_up_file.json # create new object from existing attributes

In [ ]:
!jq '.employees[] | {name: (.firstName + " " + .lastName)}' made_up_file.json # create new object from combining existing attributes

## Download using NASA near earth objects API

The near earth objects API contains the following endpoints:

- Browse: Browse the overall Asteroid data-set  
  Example query: `https://api.nasa.gov/neo/rest/v1/neo/browse?api_key=DEMO_KEY`
- Lookup: Lookup a specific Asteroid based on its NASA JPL small body (SPK-ID) ID.  
  Example query: `https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key=DEMO_KEY`
- Feed: Retrieve a list of Asteroids based on their closest approach date to Earth.  
  Example query: `https://api.nasa.gov/neo/rest/v1/feed?start_date=START_DATE&end_date=END_DATE&api_key=API_KEY`

### The `browse` endpoint

Inspect `links` and `page` attributes. The `page` attribute shows the number of `elements` accessible through the `total_pages` when viweing 20 (`size`) results at a time. The `links` attribute contains links to the current set of results (`self`) and the `next` page of results.

Let's automate downloading 10 pages of results. Before we start downloading 1657 pages of results, we want to make sure we will not hit any rate limit. What is the [rate limit for this API](https://api.nasa.gov)?

To stay well below the rate limit, we will automate downloading 10 pages of results and save them to separate files.

Create a list of links for the first 10 pages by replacing the page number in the following URL:  
`https://api.nasa.gov/neo/rest/v1/neo/browse?page=0&size=20&api_key=`.

In [ ]:
url_template = 'https://api.nasa.gov/neo/rest/v1/neo/browse?page=0&size=20&api_key='

urls = []
for page_num in range(0, 10):
  url = url_template.replace('page=0', ... + str(...)) # replace the ... with the correct page number
  urls += [url + my_api_key]
 
urls # should be a list of 10 URLs

In [ ]:
grader.check("nasa_api")

<!-- BEGIN QUESTION -->

Download from the 10 urls created in the previous question. Save them to files as shown in the code.

In [ ]:
for page_num, url in enumerate(urls):
  print('processing:', url)
  !curl -s '...' > browse_page_{page_num}.json # the query URL goes here

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Run and inspect the following shell code and explain what `xargs` does. What does `wc` tell you about each file?

_Type your answer here, replacing this text._

In [ ]:
!ls -1 browse_page_*.json | xargs wc

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

Complete the following code to collect all `near_earth_objects` arrays into one single file `asteroids_all.json`

In [ ]:
!ls -1 browse_page_*.json | xargs cat | jq -c '...' > asteroids_all.json # the jq command goes here

# keep below lines
!wc asteroids_all.json
!head -n 20 asteroids_all.json > asteroids_20.json # create a small file for testing

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Near Earth Asteroid Data

We can use `jq` directly on a json file: e.g., 
```
jq '.name' asteroids_20.json
```

Synthesize a new asteroid data from `asteroids_20.json`. The new data will consist of the following:

- `name`: asteroid name
- `id`: asteroid ID
- `diameter`: maximum estimated diameter of asteroid in kilometers
- `hazardous`: whether asteroid is potentially hazardous
- `approach_dates`: list of all `close_approach_date` in `close_approach_data`: e.g., `["1900-12-27","1907-11-05",...]`

Use the compact mode to put one record on one output row (refer to `jq --help`). Save the result to a file named `reformatted_20.json`.

First few lines of `reformatted_20.json` looks like this: 
```
[
  {
    "name": "433 Eros (A898 PA)",
    "id": "2000433",
    "diameter": 49.2084832235,
    "hazardous": false,
    "close_approach_date": [
      "1900-12-27",
      "1907-11-05",
      "1917-04-20",
```

In [ ]:
!jq ... '{name: ..., id: ..., diameter: ..., hazardous:..., close_approach_date:[...]}' asteroids_20.json | jq -s > reformatted_20.json # synthesize the output by replacing all ...'s

<!-- END QUESTION -->

## Reading JSON data with Pandas

Study the following code.

In [ ]:
import pandas as pd

# read in the reformatted JSON file
asteroids_20 = \
  pd.read_json('reformatted_20.json')\
    .astype({'id': 'category'})

# dataframe with asteroid information
asteroids_20_info = \
  asteroids_20\
    .loc[:, ['name', 'id', 'diameter', 'hazardous']]\
    .set_index('id')

# dataframe listing all close approach dates
asteroids_20_dates = \
  asteroids_20\
    .loc[:, ['id','close_approach_date']]\
    .explode('close_approach_date')\
    .astype({'close_approach_date': 'datetime64[s]'})

# add a column with the number of days since the last close approach
asteroids_20_dates['days_since_last'] = \
  asteroids_20_dates\
    .groupby('id', observed=False)['close_approach_date']\
    .transform(lambda x: x.sort_values().diff().dt.days.astype('Int64'))

# set multi-level index
asteroids_20_dates = \
  asteroids_20_dates\
    .set_index(['id','close_approach_date'])

First few rows of `asteroids_20` dataframe:

```
                     name       id   diameter  hazardous  \
0      433 Eros (A898 PA)  2000433  49.208483      False   
1    719 Albert (A911 TB)  2000719   4.529393      False   
2    887 Alinda (A918 AA)  2000887   9.954971      False   
3  1036 Ganymed (A924 UB)  2001036  83.567994      False   
4    1221 Amor (1932 EA1)  2001221   1.995446      False   

                                 close_approach_date  
0  [1900-12-27, 1907-11-05, 1917-04-20, 1924-03-0...  
1  [1909-08-21, 1911-09-08, 1941-09-08, 1956-09-1...  
2  [1910-01-04, 1914-01-02, 1918-01-03, 1922-01-0...  
3  [1910-02-25, 1911-10-15, 1924-10-17, 1998-10-1...  
4  [1900-03-08, 1908-03-14, 1916-03-17, 1924-03-2...  
```

First few rows of `asteroids_20_info` dataframe:

```
                           name   diameter  hazardous
id                                                   
2000433      433 Eros (A898 PA)  49.208483      False
2000719    719 Albert (A911 TB)   4.529393      False
2000887    887 Alinda (A918 AA)   9.954971      False
2001036  1036 Ganymed (A924 UB)  83.567994      False
2001221    1221 Amor (1932 EA1)   1.995446      False
```                           

First few rows of `asteroids_20_dates` dataframe:

```
                             days_since_last
id      close_approach_date                 
2000433 1900-12-27                      <NA>
        1907-11-05                      2504
        1917-04-20                      3454
        1924-03-05                      2511
        1931-01-30                      2522
```

## Potentially Hazardous Asteroid 2023

Filter for potentially hazardous asteroids in 2025 from `asteroids_20.json`

In [ ]:
# filter for hazardous asteroids
haz_20 = asteroids_20_info.query('hazardous == True').index.values

# subset dates dataframe to only include hazardous asteroids
haz_20_dates = asteroids_20_dates.loc[haz_20]

# filter for close approach dates in 2023
idx = pd.IndexSlice
# filter dates between '2025-01-01' and '2025-12-31'
haz_20_date_2025 = haz_20_dates.loc[..., :] # replace ... with multi-level index

In [ ]:
grader.check("hazardous_asteroids_2023")

## Read from All Files

Now, we are ready to combine all 10 query results files (we saved file names in variable called `files`). Loop through each file name, and apply the constructed `jq` command to concatenate all rows.

In [ ]:
!jq ... '{name: ..., id: ..., diameter: ..., hazardous:..., close_approach_date:[...]}' asteroids_all.json | jq -s > reformatted_all.json # synthesize the output by replacing all ...'s

# read in the reformatted JSON file
asteroids_all = ...

# dataframe with asteroid information
asteroids_all_info = ...

# dataframe listing all close approach dates
asteroids_all_dates = ...

# add a column with the number of days since the last close approach
asteroids_all_dates['days_since_last'] = ...

# set multi-level index
asteroids_all_dates = ...

In [ ]:
grader.check("asteroids_all")

In [ ]:
# filter for hazardous asteroids
haz_all = asteroids_all_info.query('hazardous == True').index.values

# subset dates dataframe to only include hazardous asteroids
haz_all_dates = asteroids_all_dates.loc[haz_all]

# filter for close approach dates in 2025
haz_all_dates.loc[idx[:, '2025-01-01':'2025-12-31'], :]

# filter dates between '2025-01-01' and '2025-12-31'
haz_all_date_2025 = haz_all_dates.loc[..., :] # replace ... with multi-level index

In [ ]:
grader.check("hazardous_asteroids_2025")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Download the zip file and submit to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)